Se importan las bibliotecas a usar. 

In [3]:
import pandas as pd
import requests 
import sys

https://stats.oecd.org/

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CRI+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LVA+LTU+LUX+MEX+NLD+NZL+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+USA+EA19+EU27_2020+G-7+NAFTA+OECDE+G-20+OECD+ARG+BRA+BGR+CHN+HRV+IND+IDN+ROU+RUS+SAU+ZAF.B1_GE+P31S14_S15+P3S13+P51+P52_P53+B11+P6+P7.GYSA+GPSA.Q/all?startTime=2020-Q4&endTime=2022-Q4

https://stats.oecd.org/SDMX-JSON/data/SNA_TABLE1/AUS+AUT+BEL+CAN+CHL+COL+CRI+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LVA+LTU+LUX+MEX+NLD+NZL+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+USA+EA19+EU27_2020+OECD+NMEC+ARG+BRA+BGR+CHN+HRV+CYP+IND+IDN+MLT+ROU+RUS+SAU+ZAF./all?startTime=2015&endTime=2021B1_GA+B1G_P119+B1G+B1GA_B+B1GC_E+B1GD+B1GF+B1GG_I+B1GJ_K+B1GL_P+B1GVA+B1GVB_E+B1GVC+B1GVF+B1GVG_I+B1GVJ+B1GVK+B1GVL+B1GVM_N+B1GVO_Q+B1GVR_U+P119+D21_D31+D21S1+D31S1+DB1_GA+B1_GE+P3_P5+P3+P31S14_S15+P31S14+P31S15+P3S13+P31S13+P32S13+P41+P5+P51+P51N1111+P51N1112+P51N1113+P51N11131+P51N11132+P51N1113I+P51N1113O+P51N1114+P51N112+P52_P53+P52+P53+B11+P6+P61+P62+P7+P71+P72+DB1_GE+B1_GI+D1+D1A_B+D1C_E+D1D+D1F+D1G_I+D1J_K+D1L_P+D1VA+D1VB_E+D1VC+D1VF+D1VG_I+D1VJ+D1VK+D1VL+D1VM_N+D1VO_Q+D1VR_U+D11+D11A_B+D11C_E+D11D+D11F+D11G_I+D11J_K+D11L_P+D11VA+D11VB_E+D11VC+D11VF+D11VG_I+D11VJ+D11VK+D11VL+D11VM_N+D11VO_Q+D11VR_U+D12+D12VA+D12VB_E+D12VC+D12VF+D12VG_I+D12VJ+D12VK+D12VL+D12VM_N+D12VO_Q+D12VR_U+B2G_B3G+D2_D3+D2S1+D3S1+DB1_GI.C+V+VP+VOB+CXC+VXCOB+VXVOB+XVP+CPC+VPCOB+VPVOB+PVP+HCXC+HVXVOB+HCPC+HVPVOB+HCPIXOE+HVPIXOE+VIXOB+DOB+G

In [314]:
datasetIdentifier ='QNA'
datos = 'B1_GE+P31S14_S15+P3S13+P51+P52_P53+B11+P6+P7.GYSA+GPSA.Q'
startTime = '2015-Q1'
endTime = '2021-Q4'

countries = {
    "AUS": "Australia",
    "AUT": "Austria",
    "CAN": "Canada",
    "CHL": "Chile"
}

#### Funcion:

In [311]:
def api_oecd(identifier, countries, datos, fechaini, fechafin, periodicidad):
    """Se importa información económica de la OECD
    
    Parametros
    ----------
    identifier: str
        Código identificador de la base de datos ???
    countries: dict
        Códigos y nombres de los países (keys, values)
    datos: str
        Código de las series
    fechaini: str
        Fecha de inicio (yyyy-qq) 
    fechafin: str
        Fecha de fin (yyyy-qq) 
    periodicidad: str
        Frecuencia de los datos: "Q", "Y"    
    
    
    Retorno
    -------
    df: pd.DataFrame
        Series consultadas
    
    
    Documentación
    -------
    https://stats.oecd.org/
    
    
    @author: Andrei Romero, Mauricio Alvarado"""

    # Uno los codigos de los paises con un '+'
    filters = ["+".join(countries.keys()) for i in countries.keys()][0]
    
    # Nombres de los países
    nombres = list(countries.values())
    
    # Cuantos paises hay?
    cantidad_paises = len(countries.keys())
    
    url = f"https://stats.oecd.org/SDMX-JSON/data/{identifier}/{filters}.{datos}/all?startTime={fechaini}&endTime={fechafin}"

    # Requests a la base de datos
    r = requests.get(url)
    if r.status_code == 200:
        pass
    else:
        print("Porfavor, revisa los datos ingresados.")
        sys.exit()


    # Consigo la ruta donde están las series, pero contiene más info
    series = r.json()["dataSets"][0]["series"]    
    
    df = pd.DataFrame()
    j = 0 # Para asignar los nombres a las columnas

    
    # Loop para consultar las series de cada país
    for i in range(0, cantidad_paises):
        fechas = list(list(series.values())[i]["observations"].keys())    # Índices con fechas
        valores = list(list(series.values())[i]["observations"].values()) # Valores, en bruto
        
        # Extraer los valores, limpios -> renombrado como 'nv'
        nv = []
        for i in range(0, len(fechas)):
            nv.append(valores[i][0])
        
        # Merge los datos de cada país
        if df.empty == True:
            df = pd.DataFrame({"fechas": fechas, nombres[j]: nv})
        else:
            df2 = pd.DataFrame({"fechas": fechas, nombres[j]: nv})
            df = df.merge(df2, how="inner")
        j += 1    
      
    # Asigno a la columna fechas en el índice  
    df = df.set_index("fechas")
    
    # Creo una lista con el rango de `fechaini` - `fechafin``
    df.index = pd.period_range(fechaini, fechafin, freq=periodicidad)

    return df

In [313]:
dta = api_oecd(datasetIdentifier, countries, datos, startTime, endTime, periodicidad="Q")
dta

,Australia,Austria,Canada,Chile
2015Q1,0.908580,2.221227,0.225043,0.895594
2015Q2,0.085701,1.823724,0.143872,0.540502
2015Q3,1.031413,2.420028,0.700350,1.219137
2015Q4,0.612914,2.662133,0.156560,1.230415
2016Q1,0.885020,2.638163,0.733006,1.743473
2016Q2,0.630758,3.197120,0.348031,1.950893
2016Q3,0.099068,2.244790,0.490214,1.738147
2016Q4,1.053592,2.692615,0.922412,2.516092
2017Q1,0.314683,2.112060,0.289430,2.064664
2017Q2,0.602469,2.083355,0.711478,2.434328
